# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

We start by loading spaCy:

In [1]:
import spacy

nlp = spacy.load('en_core_web_sm')

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [2]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(' '))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [3]:
import bz2
import csv
import pandas as pd

with bz2.open('ner-train.tsv.bz2', 'rt') as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open('ner-dev.tsv.bz2', 'rt') as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [4]:
df_train.head()

sentence_id                                          sentence  beg  end  \
0    0000-000  EU rejects German call to boycott British lamb .    0    1   
1    0000-000  EU rejects German call to boycott British lamb .    2    3   
2    0000-000  EU rejects German call to boycott British lamb .    6    7   
3    0000-001                                   Peter Blackburn    0    2   
4    0000-002                               BRUSSELS 1996-08-22    0    1   

            label  
0         --NME--  
1         Germany  
2  United_Kingdom  
3         --NME--  
4        Brussels

In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [5]:
def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.
    
    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    # TODO: Replace the next line with your own code
    gold = list(gold)
    pred = list(pred)
    
    precision_counter = 0
    recall_counter = 0
    for i, value in enumerate(gold):
      if gold[i] in pred:
        precision_counter+=1

    for i, value in enumerate(pred):
      if pred[i] in gold:
        recall_counter+=1

    precision = precision_counter/len(pred) 
    recall = recall_counter/len(gold)
    f1_score = 2 * precision * recall / (precision + recall)
    f1_score = round(f1_score, 2)

    print(f'The precision is: {precision} \n The recall is: {recall} \n The F1-value is: {f1_score}')

To test your code, you can run the following cell:

In [6]:
evaluation_report(set(range(3)), set(range(5)))

The precision is: 0.6 
 The recall is: 1.0 
 The F1-value is: 0.75


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [7]:
df_dev.head(10)

sentence_id                                           sentence  beg  end  \
0    0946-000  CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...    2    3   
1    0946-001                                  LONDON 1996-08-30    0    1   
2    0946-002  West Indian all-rounder Phil Simmons took four...    0    2   
3    0946-002  West Indian all-rounder Phil Simmons took four...    3    5   
4    0946-002  West Indian all-rounder Phil Simmons took four...   12   13   
5    0946-002  West Indian all-rounder Phil Simmons took four...   14   15   
6    0946-003  Their stay on top , though , may be short-live...   13   14   
7    0946-003  Their stay on top , though , may be short-live...   15   16   
8    0946-003  Their stay on top , though , may be short-live...   17   18   
9    0946-003  Their stay on top , though , may be short-live...   24   25   

                                label  
0  Leicestershire_County_Cricket_Club  
1                              London  
2            West_Indies_cricket_team  
3                        Phil_Simmons  
4  Leicestershire_County_Cricket_Club  
5        Somerset_County_Cricket_Club  
6           Essex_County_Cricket_Club  
7      Derbyshire_County_Cricket_Club  
8          Surrey_County_Cricket_Club  
9            Kent_County_Cricket_Club

In [8]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        yield row[1], row[3], row[4] # yield tuples
#     yield None

In [9]:
span_test = gold_spans(df_dev)
len(list(span_test)) # Unique triples


5917

In [10]:
for i in gold_spans(df_dev):
    print(i)

('0946-000', 2, 3)
('0946-001', 0, 1)
('0946-002', 0, 2)
('0946-002', 3, 5)
('0946-002', 12, 13)
('0946-002', 14, 15)
('0946-003', 13, 14)
('0946-003', 15, 16)
('0946-003', 17, 18)
('0946-003', 24, 25)
('0946-003', 35, 36)
('0946-004', 2, 3)
('0946-004', 11, 13)
('0946-004', 14, 15)
('0946-004', 29, 30)
('0946-004', 31, 33)
('0946-005', 4, 5)
('0946-005', 14, 15)
('0946-006', 0, 1)
('0946-006', 12, 14)
('0946-006', 15, 17)
('0946-006', 26, 27)
('0946-006', 28, 29)
('0946-007', 0, 1)
('0946-007', 5, 6)
('0946-007', 22, 23)
('0946-008', 3, 4)
('0946-008', 13, 14)
('0946-009', 2, 3)
('0946-009', 4, 5)
('0946-009', 6, 8)
('0946-009', 13, 14)
('0946-009', 39, 40)
('0946-010', 5, 6)
('0946-010', 7, 9)
('0946-010', 13, 14)
('0946-011', 0, 1)
('0946-011', 14, 15)
('0946-012', 0, 1)
('0946-012', 1, 3)
('0946-012', 8, 10)
('0946-012', 14, 16)
('0946-012', 20, 21)
('0946-013', 17, 18)
('0946-013', 24, 25)
('0946-014', 9, 11)
('0946-014', 12, 13)
('0946-014', 15, 17)
('0946-015', 2, 3)
('0947-000'

('0966-137', 5, 6)
('0966-138', 1, 3)
('0966-138', 4, 5)
('0966-139', 1, 3)
('0966-139', 4, 5)
('0966-140', 1, 3)
('0966-140', 4, 5)
('0966-141', 1, 3)
('0966-141', 4, 5)
('0966-143', 1, 3)
('0966-143', 4, 5)
('0966-144', 1, 3)
('0966-144', 4, 5)
('0966-145', 1, 3)
('0966-145', 4, 5)
('0966-146', 1, 3)
('0966-146', 4, 5)
('0966-147', 1, 3)
('0966-147', 4, 5)
('0966-148', 1, 3)
('0966-148', 4, 5)
('0966-149', 1, 3)
('0966-149', 4, 5)
('0966-150', 1, 3)
('0966-150', 4, 5)
('0966-152', 1, 3)
('0966-152', 4, 5)
('0966-153', 1, 3)
('0966-153', 4, 5)
('0966-154', 1, 3)
('0966-154', 4, 5)
('0966-155', 1, 3)
('0966-155', 4, 5)
('0966-156', 1, 3)
('0966-156', 4, 5)
('0966-157', 1, 3)
('0966-157', 4, 5)
('0966-158', 1, 3)
('0966-158', 4, 5)
('0966-159', 1, 3)
('0966-159', 4, 5)
('0966-160', 4, 6)
('0966-161', 1, 3)
('0966-161', 4, 5)
('0966-161', 7, 9)
('0966-161', 10, 11)
('0966-161', 13, 15)
('0966-161', 16, 17)
('0966-161', 19, 21)
('0966-161', 22, 23)
('0966-162', 1, 3)
('0966-162', 4, 5)
('

('1010-020', 17, 18)
('1010-021', 20, 22)
('1011-000', 0, 2)
('1011-001', 0, 2)
('1011-002', 0, 1)
('1011-003', 7, 11)
('1011-003', 12, 14)
('1011-003', 22, 23)
('1011-004', 1, 2)
('1011-004', 6, 7)
('1011-004', 19, 21)
('1011-004', 26, 31)
('1011-005', 8, 9)
('1011-005', 15, 16)
('1011-005', 17, 18)
('1011-006', 0, 2)
('1011-006', 2, 5)
('1011-006', 26, 27)
('1011-006', 31, 33)
('1011-007', 0, 2)
('1011-007', 3, 4)
('1011-007', 4, 7)
('1011-007', 28, 29)
('1011-008', 6, 7)
('1011-008', 8, 9)
('1011-008', 10, 11)
('1011-008', 14, 16)
('1011-008', 19, 23)
('1011-009', 0, 4)
('1011-009', 9, 12)
('1011-010', 10, 11)
('1011-010', 13, 14)
('1011-011', 0, 1)
('1011-011', 2, 3)
('1011-012', 10, 11)
('1011-012', 12, 13)
('1011-012', 13, 17)
('1011-012', 18, 20)
('1011-012', 21, 22)
('1011-012', 22, 23)
('1011-012', 24, 27)
('1011-012', 29, 33)
('1011-012', 34, 35)
('1011-013', 2, 4)
('1011-013', 5, 6)
('1011-013', 7, 9)
('1011-013', 10, 12)
('1011-014', 0, 1)
('1011-016', 0, 2)
('1011-016', 6,

('1054-019', 6, 7)
('1054-020', 1, 3)
('1054-020', 4, 5)
('1054-020', 6, 7)
('1054-021', 1, 2)
('1054-022', 1, 2)
('1054-023', 1, 4)
('1054-023', 5, 6)
('1054-023', 7, 8)
('1054-024', 1, 2)
('1054-025', 1, 2)
('1055-000', 2, 3)
('1055-000', 4, 5)
('1055-001', 0, 1)
('1055-001', 2, 3)
('1055-002', 8, 9)
('1055-002', 10, 11)
('1055-003', 0, 1)
('1055-004', 0, 2)
('1055-004', 3, 5)
('1055-004', 6, 8)
('1055-005', 0, 2)
('1055-005', 3, 5)
('1055-006', 0, 2)
('1055-006', 4, 6)
('1055-007', 0, 2)
('1055-007', 4, 5)
('1055-008', 0, 2)
('1055-009', 0, 2)
('1055-010', 0, 2)
('1055-011', 0, 2)
('1055-012', 0, 2)
('1055-012', 3, 5)
('1055-013', 0, 2)
('1055-017', 4, 6)
('1055-018', 2, 4)
('1055-018', 6, 8)
('1055-019', 0, 1)
('1055-019', 3, 5)
('1055-019', 7, 9)
('1055-020', 2, 4)
('1055-022', 0, 2)
('1055-022', 3, 4)
('1055-022', 5, 6)
('1055-023', 0, 2)
('1055-023', 3, 4)
('1055-023', 5, 6)
('1055-024', 0, 2)
('1055-024', 4, 5)
('1055-025', 0, 2)
('1055-025', 3, 4)
('1055-026', 0, 1)
('1055-026

('1077-006', 0, 1)
('1077-006', 2, 3)
('1077-006', 5, 7)
('1078-000', 2, 5)
('1078-001', 0, 1)
('1078-002', 3, 6)
('1078-005', 0, 1)
('1078-005', 2, 3)
('1078-005', 5, 6)
('1078-005', 7, 8)
('1078-007', 0, 1)
('1078-007', 2, 3)
('1078-007', 5, 7)
('1078-007', 8, 9)
('1079-000', 2, 5)
('1079-001', 0, 1)
('1079-002', 3, 6)
('1079-005', 0, 1)
('1079-005', 2, 3)
('1079-005', 5, 6)
('1079-005', 7, 8)
('1079-007', 0, 1)
('1079-007', 2, 3)
('1079-007', 5, 7)
('1079-007', 8, 9)
('1080-000', 2, 3)
('1080-000', 4, 5)
('1080-000', 6, 8)
('1080-001', 0, 1)
('1080-002', 0, 1)
('1080-002', 2, 3)
('1080-002', 10, 12)
('1080-002', 13, 14)
('1080-004', 0, 1)
('1080-004', 2, 4)
('1080-004', 9, 11)
('1080-004', 15, 17)
('1081-000', 2, 3)
('1081-000', 4, 5)
('1081-000', 8, 10)
('1081-001', 0, 1)
('1081-002', 0, 1)
('1081-002', 2, 3)
('1081-002', 7, 9)
('1081-002', 10, 11)
('1082-000', 2, 3)
('1082-000', 4, 5)
('1082-000', 6, 8)
('1082-001', 0, 1)
('1082-002', 0, 1)
('1082-002', 2, 3)
('1082-002', 10, 12)


('1139-004', 4, 5)
('1139-004', 17, 18)
('1139-005', 0, 1)
('1139-006', 3, 4)
('1139-006', 13, 15)
('1139-008', 0, 1)
('1139-008', 20, 21)
('1139-008', 22, 24)
('1140-000', 0, 2)
('1140-001', 0, 2)
('1140-002', 11, 13)
('1140-003', 12, 14)
('1140-003', 16, 18)
('1140-003', 24, 27)
('1140-005', 0, 1)
('1140-005', 8, 9)
('1140-005', 15, 16)
('1140-005', 17, 18)
('1141-000', 0, 1)
('1141-001', 0, 1)
('1141-002', 1, 2)
('1141-002', 21, 23)
('1141-003', 2, 3)
('1141-003', 10, 11)
('1141-006', 0, 1)
('1141-007', 3, 4)
('1141-007', 11, 12)
('1141-008', 0, 1)
('1142-000', 0, 2)
('1142-000', 3, 4)
('1142-000', 7, 8)
('1142-001', 0, 1)
('1142-002', 0, 1)
('1142-002', 5, 6)
('1142-002', 7, 8)
('1142-002', 16, 17)
('1142-002', 22, 24)
('1142-002', 26, 27)
('1142-005', 6, 7)
('1142-005', 9, 11)
('1142-005', 13, 16)
('1142-005', 17, 18)
('1142-005', 20, 21)
('1142-005', 24, 25)
('1142-006', 0, 1)
('1142-006', 12, 13)
('1142-006', 16, 17)
('1142-006', 20, 21)
('1142-006', 32, 33)
('1142-008', 0, 1)
(

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [11]:
spans_dev_gold = set(gold_spans(df_dev))
print(len(spans_dev_gold))

5917


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [12]:
# TODO: Write code here to run and evaluate the spaCy NER on the development data

def spacy_entity(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        doc = nlp(row[2])
        for ent in doc.ents:
            yield row[1], ent.start, ent.end


In [13]:
dev_spacy_pred = set(spacy_entity(df_dev))

In [14]:
evaluation_report(spans_dev_gold, dev_spacy_pred)

The precision is: 0.5383185017930668 
 The recall is: 0.6849754943383471 
 The F1-value is: 0.6


In [15]:
spans_dev_gold

{('1076-043', 0, 1),
 ('1152-003', 3, 4),
 ('1002-001', 2, 3),
 ('1130-005', 62, 64),
 ('1127-021', 3, 4),
 ('0954-000', 2, 5),
 ('1155-009', 15, 16),
 ('1030-039', 2, 3),
 ('1058-007', 0, 1),
 ('1073-008', 2, 4),
 ('0974-011', 0, 2),
 ('1005-015', 7, 9),
 ('1101-012', 21, 23),
 ('0966-132', 1, 3),
 ('0965-008', 8, 9),
 ('1127-008', 18, 19),
 ('1116-025', 4, 6),
 ('1030-023', 11, 13),
 ('1048-002', 2, 3),
 ('1068-037', 2, 3),
 ('0957-017', 3, 5),
 ('0984-016', 18, 19),
 ('1097-006', 17, 19),
 ('0981-007', 1, 4),
 ('1036-034', 0, 1),
 ('1031-014', 30, 31),
 ('1073-004', 13, 15),
 ('1017-003', 21, 23),
 ('1069-009', 3, 4),
 ('1096-036', 13, 15),
 ('1039-011', 24, 28),
 ('1120-005', 12, 13),
 ('0956-006', 2, 3),
 ('1018-002', 0, 1),
 ('0966-162', 13, 15),
 ('1073-005', 2, 4),
 ('1051-010', 32, 34),
 ('0994-015', 10, 11),
 ('1136-006', 0, 3),
 ('0966-161', 22, 23),
 ('1103-016', 32, 34),
 ('1009-008', 10, 11),
 ('1058-002', 17, 19),
 ('1006-011', 0, 1),
 ('1009-018', 27, 29),
 ('1078-002',

## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Here is a function that prints the false positives as well as the false negatives spans for a data frame, given a reference set of gold-standard spans and a candidate set of predicted spans.

In [16]:
from collections import defaultdict

def error_report(df, spans_gold, spans_pred):
    false_pos = defaultdict(list)
    for s, b, e in spans_pred - spans_gold:
        false_pos[s].append((b, e))
    false_neg = defaultdict(list)
    for s, b, e in spans_gold - spans_pred:
        false_neg[s].append((b, e))
    for row in df.drop_duplicates('sentence_id').itertuples():
        if row.sentence_id in false_pos or row.sentence_id in false_neg:
            print('Sentence:', row.sentence)
            for b, e in false_pos[row.sentence_id]:
                print('  FP:', ' '.join(row.sentence.split()[b:e]))
            for b, e in false_neg[row.sentence_id]:
                print('  FN:', ' '.join(row.sentence.split()[b:e]))

Use this function to inspect and analyse the errors that the automated prediction makes. Can you see any patterns? Base your analysis on the first 500 rows of the training data. Summarize your observations in a short text.

In [17]:
# TODO: Write code here to do your analysis
error_report(df_dev, spans_dev_gold, dev_spacy_pred)

Sentence: CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .
  FN: LEICESTERSHIRE
Sentence: West Indian all-rounder Phil Simmons took four for 38 on Friday as Leicestershire beat Somerset by an innings and 39 runs in two days to take over at the head of the county championship .
  FP: four
  FP: 38
  FP: Friday
  FP: 39
  FP: two days
  FN: Somerset
Sentence: After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 .
  FP: 83
  FP: 83
  FP: 94
  FP: first
  FP: the opening morning
  FP: 296
  FP: three
Sentence: Trailing by 213 , Somerset got a solid start to their second innings before Simmons stepped in to bundle them out for 174 .
  FP: second
  FP: 174
  FP: 213
Sentence: Essex , however , look certain to regain their top spot after Nasser Hussain and Peter Such gave them a firm grip on their match against Yorksh

Sentence: OB 42 6 62 .409 20 1/2
  FP: 20 1/2
  FP: 42
  FN: OB
Sentence: TENNIS - FRIDAY 'S RESULTS FROM THE U.S. OPEN .
  FP: U.S.
  FN: U.S. OPEN
Sentence: NEW YORK 1996-08-30
  FP: NEW YORK 1996-08-30
  FN: NEW YORK
Sentence: Results from the U.S. Open Tennis Championships at the National Tennis Centre on Friday ( prefix number denotes seeding ) :
  FP: Friday
  FP: U.S.
  FP: the National Tennis Centre
  FN: U.S. Open Tennis Championships
  FN: National Tennis Centre
Sentence: Sandrine Testud ( France ) beat Ines Gorrochategui ( Argentina ) 4-6 6-2 6-1
  FP: 6-2
  FN: Ines Gorrochategui
Sentence: 4 - Goran Ivanisevic ( Croatia ) beat Scott Draper ( Australia ) 6-7 ( 1-7 ) 6-3 6-4 6-4
  FP: 6-7
  FN: Croatia
  FN: Australia
  FN: Goran Ivanisevic
Sentence: Mark Philippoussis ( Australia ) beat Andrei Olhovskiy ( Russia ) 6 - 3 6-4 6-2
  FP: 6 - 3
Sentence: Sjeng Schalken ( Netherlands ) beat David Rikl ( Czech Republic ) 6 - 2 6-4 6-4
  FP: Czech
  FN: Czech Republic
Sentence: Guy 

  FN: Torrance Zellner
Sentence: 2. Samuel Matete ( Zambia ) 48.34
  FP: 2.
  FP: 48.34
Sentence: 3. Derrick Adkins ( U.S. ) 48.62
  FP: 48.62
  FP: 3.
Sentence: 4. Fabrizio Mori ( Italy ) 49.21
  FP: 49.21
  FP: 4.
Sentence: 5. Sven Nylander ( Sweden ) 49.22
  FP: 49.22
  FP: 5.
  FN: Sven Nylander
Sentence: 6. Eric Thomas ( U.S. ) 49.35
  FP: 49.35
Sentence: 7. Rohan Robinson ( Australia ) 49.36
  FP: 49.36
  FP: 7.
Sentence: 8. Dusan Kovacs ( Hungary ) 49.58
  FP: 49.58
Sentence: 1. Falilat Ogunkoya ( Nigeria ) 50.31 seconds
  FP: 50.31 seconds
  FP: 1.
  FP: Falilat Ogunkoya ( Nigeria
  FN: Nigeria
  FN: Falilat Ogunkoya
Sentence: 2. Jearl Miles ( U.S. ) 50.42
  FP: 2. Jearl Miles
  FP: 50.42
  FN: Jearl Miles
Sentence: 3. Fatima Yusuf ( Nigeria ) 51.43
  FP: 51.43
  FP: 3.
Sentence: 4. Anja Ruecker ( Germany ) 51.61
  FP: 4.
  FP: 51.61
  FN: Anja Ruecker
Sentence: 5. Olabisi Afolabi ( Nigeria ) 51.98
  FP: Olabisi Afolabi ( Nigeria
  FP: 51.98
  FP: 5.
  FN: Olabisi Afolabi
  FN:

  FP: monthly
  FP: December 1993
  FP: 8.8
  FP: August
Sentence: Primary dealers immediately sold Eurodollar and bond futures , after the market on average was expecting the index to rise marginally to 51.9 from July 's 51.2 .
  FP: 51.9
  FP: July
  FP: 51.2
  FN: Eurodollar
Sentence: Traders also said Japanese investors were unwinding long Eurodollar futures / short swaps , and that heavy put buying helped pressure Eurodollars to lower levels before the close .
  FN: Eurodollar
  FN: Eurodollars
Sentence: One U.S. firm bought 35,000 September 97 mid-curve put options at a strike price of 93.25 to 93.30 in the last two sessions , while a French firm bought 4,000 September 93.30 to 93.32 put spreads .
  FP: 93.25
  FP: 93.30
  FP: 4,000
  FP: September 97
  FP: two
  FP: 93.32
  FP: 93.30
  FP: 35,000
  FP: One
Sentence: Rumors circulated that the Federal Reserve was buying five-year notes , and that a renowned hedge fund manager was buying 10-year notes in the cash markets .
  FP: t

  FP: November
Sentence: On Friday , the Dow Jones index closed down 31.44 points at 5,616.21 .
  FP: 31.44
  FP: Friday
  FP: 5,616.21
Sentence: The Nasdaq composite index closed 3.53 points lower Friday at 1,141.50 .
  FP: Friday
  FP: 1,141.50
  FP: 3.53
Sentence: The Standard & Poor 's index of 500 stocks was off 5.41 points at 651.99 , down 15.03 points for the week .
  FP: 5.41
  FP: 500
  FP: 15.03
  FP: the week
  FP: 651.99
  FN: Standard & Poor
Sentence: The American Stock Exchange index was down 1.66 points at 559.68 , and was off 1.26 for the week .
  FP: the week
  FP: 1.66
  FP: The American Stock Exchange
  FP: 559.68
  FP: 1.26
  FN: American Stock Exchange
Sentence: WASHINGTON 1996-08-30
  FP: 1996-08-30
Sentence: Factory orders rose in July and manufacturing surged in the Midwest in August , reports said Friday , sparking worries about inflation that battered financial markets for a second straight day .
  FP: Friday
  FP: July
  FP: a second straight day
  FP: August

...

Sentence: Cardiff 4 2 1 1 3 2 7
  FP: 1 1 3 2 7
  FN: Cardiff
Sentence: Scunthorpe 4 2 1 1 3 3 7
  FP: Scunthorpe 4 2 1 1 3 3 7
  FN: Scunthorpe
Sentence: Carlisle 4 2 1 1 2 1 7
  FN: Carlisle
Sentence: Scarborough 4 1 3 0 5 3 6
  FP: 4 1 3
  FP: 0 5 3 6
  FN: Scarborough
Sentence: Barnet 4 1 2 1 4 2 5
  FN: Barnet
Sentence: Exeter 4 1 2 1 4 5 5
  FP: 4 1 2 1 4 5 5
  FN: Exeter
Sentence: Cambridge United 4 1 2 1 3 4 5
  FP: Cambridge
  FP: 4 1 2 1 3 4 5
  FN: Cambridge United
Sentence: Chester 4 1 1 2 6 7 4
  FP: 4 1 1 2
  FN: Chester
Sentence: Doncaster 4 1 1 2 4 5 4
  FN: Doncaster
Sentence: Swansea 4 1 0 3 4 9 3
  FP: 4 1
  FP: 3 4 9 3
  FN: Swansea
Sentence: Colchester 4 0 3 1 2 4 3
  FN: Colchester
Sentence: Rochdale 4 0 2 2 2 4 2
  FN: Rochdale
Sentence: Mansfield 4 0 2 2 2 6 2
  FN: Mansfield
Sentence: SOCCER - SCOTTISH LEAGUE RESULTS .
  FP: SOCCER - SCOTTISH LEAGUE RESULTS
  FN: SCOTTISH
Sentence: GLASGOW 1996-08-31
  FN: GLASGOW
Sentence: Greenock Morton 1 Falkirk 0
  FP: Gre

  FN: LITHUANIA
  FN: ROMANIA
  FN: WORLD CUP
Sentence: BUCHAREST 1996-08-31
  FN: BUCHAREST
Sentence: Romania beat Lithuania 3-0 ( halftime 1-0 ) in a World Cup soccer European group 8 qualifier on Saturday .
  FP: 1-0
  FP: 8 qualifier
  FP: Saturday
Sentence: Romania - Viorel Moldovan ( 21st minute ) , Dan Petrescu ( 65th ) , Constantin Galca ( 77th )
  FP: 77th
  FP: Romania - Viorel Moldovan
  FP: 21st minute
  FP: 65th
  FN: Viorel Moldovan
  FN: Romania
Sentence: SOCCER - ARMENIA AND PORTUGAL DRAW 0-0 IN WORLD CUP QUALIFIER .
  FP: WORLD CUP QUALIFIER
  FN: WORLD CUP
  FN: PORTUGAL
  FN: ARMENIA
Sentence: YEREVAN 1996-08-31
  FN: YEREVAN
Sentence: Armenia and Portugal drew 0-0 in a World Cup soccer European group 9 qualifier on Saturday .
  FP: 9 qualifier
  FP: Saturday
Sentence: SOCCER - AZERBAIJAN BEAT SWITZERLAND IN WORLD CUP QUALIFIER .
  FP: WORLD CUP QUALIFIER
  FN: AZERBAIJAN
  FN: SWITZERLAND
  FN: WORLD CUP
Sentence: BAKU 1996-08-31
  FN: BAKU
Sentence: Azerbaijan beat

Sentence: Scotland : Andrew Goram , Craig Burley , Thomas Boyd , Colin Calderwood , Colin Hendry , Thomas McKinley , Duncan Ferguson , Stuart McCall , Alistair McCoist ( Gordon Durie 75th ) , Gary McAllister , John Collins .
  FP: Gordon Durie 75th
  FN: Gordon Durie
Sentence: BOXING - JOHNSON WINS UNANIMOUS POIUNTS VERDICT .
  FN: JOHNSON
Sentence: DUBLIN 1996-08-31
  FN: DUBLIN
Sentence: American Tom Johnson successfully defended his IBF featherweight title when he earned a unanimous points decision over Venezuela 's Ramon Guzman on Saturday .
  FP: Saturday
Sentence: SOCCER - FRANCE LAUNCH 1998 WORLD CUP BUILD-UP WITH 2-0 WIN .
  FP: 1998
  FN: FRANCE
Sentence: Euro 96 absentee Nicolas Ouedec and Youri Djorkaeff scored the goals as 1998 World Cup hosts France beat Mexico 2-0 in a friendly international on Saturday .
  FP: 1998
  FP: Saturday
Sentence: The victory extended to 29 matches France 's unbeaten run under coach Aime Jacquet , their Euro 96 semifinal elimination having come 

  FP: September
  FP: more than than 50 percent
  FN: Ruch
Sentence: News-stand chain Ruch , which controls about 65 percent of Poland 's press distribution market , had a net profit of 16.2 million zlotys on sales of 2.7 billion zlotys in 1995 .
  FP: 16.2 million zlotys
  FP: 1995
  FP: about 65 percent
  FP: 2.7 billion zlotys
  FN: Ruch
Sentence: Zycie Warszawy said the new , open consortium , which also included several listed Polish firms , would on Monday inform Privatisation Minister Wieslaw Kaczmarek of its plans .
  FP: Monday
Sentence: It aims to invest $ 200 million in Ruch over five years -- more than the sum Ruch says it needs to upgrade its outlets .
  FP: $ 200 million
  FP: five years
  FN: Ruch
  FN: Ruch
Sentence: Initially Poland offered up to 75 percent of Ruch but in March Kaczmarek cancelled the tender and offered a minority stake with an option to increase the equity .
  FP: up to 75 percent
  FP: March
  FN: Ruch
Sentence: Two consortia -- UWP-Hachette and a co

  FP: 55 pounds
  FP: 25 kg
  FP: 42 kg
  FP: 92 pounds
  FP: 1949
  FP: Nationalist
Sentence: Two Chinese cities are to ban the use of disposable plastic containers as part of efforts to fight pollution , the China Daily said on Saturday .
  FP: Saturday
  FP: the China Daily
  FP: Two
  FN: China Daily
Sentence: Authorities in Wuhan , capital of the central province of Hubei , would punish those who sell or use disposable plastic containers from September 1 , the newspaper said .
  FP: September 1
Sentence: Wuhan consumes more than 200 million disposable plastic containers a year , the newspaper said .
  FP: more than 200 million
Sentence: The boomtown of Guangzhou , capital of the southern province of Guangdong , would ban disposable plastic containers by the end of 1996 , it said .
  FP: the end of 1996
Sentence: Guangzhou uses up 500,000 such containers each day , the newspaper said .
  FP: 500,000
  FP: each day
Sentence: Iraqi Kurds say Iranian troops enter north Iraq .
  FP: Ir

*TODO: Write a short text that summarises the errors that you observed* <br>

**We need to remove CARDINAL , DATE, TIME and ORDINAL from the Entity**

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to post-process the output produced by spaCy. You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [18]:
# TODO: Write code here to improve the span prediction from Problem 2
def spacy_entity_improved(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        doc = nlp(row[2])
        for ent in doc.ents:
            if ent.label_ != 'CARDINAL' and ent.label_ != 'DATE' and ent.label_ != 'TIME' and ent.label_ != 'ORDINAL' and ent.label_ != 'PERCENT':
                yield row[1], ent.start, ent.end

In [19]:
dev_spacy_pred_new = set(spacy_entity_improved(df_dev))

In [20]:
evaluation_report(spans_dev_gold, dev_spacy_pred_new)

The precision is: 0.8068249850329275 
 The recall is: 0.683285448707115 
 The F1-value is: 0.74


In [21]:
error_report(df_dev, spans_dev_gold, dev_spacy_pred_new)

Sentence: CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .
  FN: LEICESTERSHIRE
Sentence: West Indian all-rounder Phil Simmons took four for 38 on Friday as Leicestershire beat Somerset by an innings and 39 runs in two days to take over at the head of the county championship .
  FN: Somerset
Sentence: Essex , however , look certain to regain their top spot after Nasser Hussain and Peter Such gave them a firm grip on their match against Yorkshire at Headingley .
  FN: Headingley
Sentence: Hussain , considered surplus to England 's one-day requirements , struck 158 , his first championship century of the season , as Essex reached 372 and took a first innings lead of 82 .
  FN: Hussain
Sentence: By the close Yorkshire had turned that into a 37-run advantage but off-spinner Such had scuttled their hopes , taking four for 24 in 48 balls and leaving them hanging on 119 for five and praying for rain .
  FP: 37-run
  FN: Such
Sentence: At the Oval , Surrey captain Chris Lewis 

  FP: the Berlin Grand Prix
  FN: Berlin Grand Prix
Sentence: 2. Ludmila Engquist ( Sweden ) 12.74
  FN: Ludmila Engquist
Sentence: 4. Brigita Bokovec ( Slovenia ) 12.92
  FP: Brigita Bokovec ( Slovenia
  FN: Brigita Bokovec
  FN: Slovenia
Sentence: 5. Leah Pells ( Canada ) 4:09.95
  FN: Leah Pells
Sentence: 3. Florian Schwarthoff ( Germany ) 13.36
  FP: Florian
  FN: Florian Schwarthoff
Sentence: 3. Ato Boldon ( Trinidad ) 20.37
  FN: Ato Boldon
Sentence: 4. Geir Moen ( Norway ) 20.41
  FN: Geir Moen
Sentence: 1. Astrid Kumbernuss ( Germany ) 19.89 metres
  FP: 19.89 metres
Sentence: 1. Noureddine Morceli ( Algeria ) 3 minutes 49.09 seconds
  FN: Noureddine Morceli
Sentence: 4. Laban Rotich ( Kenya ) 3:53.42
  FN: Laban Rotich
Sentence: 1. Lars Riedel ( Germany ) 70.60 metres
  FP: 70.60 metres
Sentence: 7. Andreas Seelig ( Germany ) 62.00
  FP: Seelig
  FN: Andreas Seelig
Sentence: 1. Gail Devers ( U.S. ) 10.89 seconds
  FN: Gail Devers
Sentence: 3. Gwen Torrence ( U.S. ) 11.07
  FP:

Sentence: Bowling : S. Waugh 5-1-36-1 , Law 2-0-23-0 , McGrath 9.5-0-44-1 ,
  FP: McGrath 9.5-0-44-1
  FN: S. Waugh
  FN: McGrath
  FN: Law
Sentence: Fleming 8-1-26-3 , Gillespie 6-0-27-0 , M. Waugh 5-0-29-0 , Lehmann
  FP: M. Waugh 5-0-29-0
  FP: Gillespie 6-0-27-0
  FN: Gillespie
  FN: Fleming
  FN: M. Waugh
Sentence: 6-0-26-0 , Bevan 4-0-18-0 .
  FP: Bevan 4-0-18-0 .
  FN: Bevan
Sentence: CRICKET - AUSTRALIA 228-9 IN 50 OVERS V SRI LANKA .
  FP: V SRI LANKA
  FN: AUSTRALIA
  FN: SRI LANKA
Sentence: Australia scored 228 for nine wickets in their 50 overs against Sri Lanka in the third day-night limited overs match of the Singer World Series tournament on Friday .
  FP: the Singer World Series
  FN: Singer World Series
Sentence: CRICKET - AUSTRALIA WIN TOSS AND CHOOSE TO BAT .
  FN: AUSTRALIA
Sentence: overs cricket match in the Singer world series tournament on
  FP: overs cricket match
  FN: Singer
Sentence: Australia - Ian Healy ( captain ) , Michael Bevan , Damien Flemming , Jason

  FN: Alexander Lebed
Sentence: " Lebed is now in Chechnya solving some problems , " Interfax quoted Chernomyrdin as saying . "
  FP: Interfax quoted Chernomyrdin
  FN: Lebed
  FN: Interfax
  FN: Chernomyrdin
Sentence: Moscow , which wants to keep Chechnya as a part of the Russian Federation , sent troops to the region in December 1994 to quell its independence bid .
  FP: the Russian Federation
  FN: Russian Federation
Sentence: Itar-Tass news agency quoted Lebed as saying that he would suggest to the rebels that the decision on Chechnya 's future political status be deferred by up to 10 years .
  FP: Itar-Tass news agency
  FN: Itar-Tass
Sentence: Lebed , Chechens sign framework political deal .
  FN: Chechens
Sentence: " We just now signed a statement and attached the basic principles of relations between the Russian Federation and the Chechen Republic , " Lebed told reporters after he and Maskhadov signed a package of documents .
  FP: the Chechen Republic
  FP: the Russian Federat

Sentence: An Israeli roadblock stopped the 38-year-old Palestinian from answering Yasser Arafat 's call to worship at Jerusalem 's al-Aqsa mosque on Friday .
  FP: Yasser Arafat 's
  FN: Yasser Arafat
  FN: al-Aqsa
Sentence: Palestinian President Arafat , attacking Israel 's decision to expand Jewish settlements and its policy on Jerusalem , went before the Palestinian legislature on Wednesday to urge the two million Arabs in the West Bank and Gaza to go to the holy city .
  FP: the West Bank
  FN: West Bank
Sentence: Palestinians have been banned by Israel from travelling from the West Bank to Jerusalem since suicide bombings by Moslem militants killed 59 people in the Jewish state in February and March .
  FP: the West Bank
  FN: West Bank
Sentence: The PLO wants Arab East Jerusalem as the capital of a future Palestinian state .
  FP: East Jerusalem
  FP: Arab
  FN: Arab East Jerusalem
Sentence: " I am heeding Abu Ammar 's ( Arafat 's ) call to pray at al-Aqsa and I came .
  FP: Abu 

Sentence: His aides said Arafat would hold the weekly meeting of the Palestinian self-rule Authority 's cabinet in Nablus on Saturday .
  FP: Palestinian
  FP: Authority
  FN: Palestinian self-rule Authority
Sentence: In Jerusalem , Israeli security forces were bracing for thousands of Palestinians expected to answer Arafat 's call earlier this week to come to the city holy to Moslems , Arabs and Jews to pray in protest against Israel 's settlement policy in the West Bank and delay in peace negotiations .
  FP: the West Bank
  FN: West Bank
Sentence: Palestinians want Arab East Jerusalem as the capital of a future independent state .
  FP: East Jerusalem
  FP: Arab
  FN: Arab East Jerusalem
Sentence: The Palestinian letter from Marwan Jilani said the destruction of the Jerusalem centre was an effort by Israel to " alter the character , demographic composition and status of the Holy City of Jerusalem " and violated agreements between Israel and the Palestinian Liberation Organisation .


Sentence: Clydebank 2 1 0 1 1 3 3
  FN: Clydebank
Sentence: Partick 3 0 2 1 1 2 2
  FN: Partick
Sentence: Stirling 3 0 1 2 1 3 1
  FN: Stirling
Sentence: East Fife 2 0 1 1 0 4 1
  FN: East Fife
Sentence: Queen of South 3 2 0 1 5 4 6
  FN: Queen of South
Sentence: Ayr 3 1 2 0 8 2 5
  FN: Ayr
Sentence: Stenhousemuir 3 1 1 1 6 1 4
  FN: Stenhousemuir
Sentence: Stranraer 3 1 1 1 3 3 4
  FN: Stranraer
Sentence: Brechin 3 0 3 0 2 2 3
  FN: Brechin
Sentence: Clyde 3 1 0 2 2 5 3
  FN: Clyde
Sentence: Berwick 3 0 0 3 1 14 0
  FN: Berwick
Sentence: Albion 3 3 0 0 5 0 9
  FN: Albion
Sentence: Forfar 3 2 0 1 7 4 6
  FN: Forfar
Sentence: Cowdenbeath 3 2 0 1 4 3 6
  FN: Cowdenbeath
Sentence: Arbroath 3 1 2 0 4 2 5
  FN: Arbroath
Sentence: Alloa 3 1 1 1 3 3 4
  FN: Alloa
Sentence: Queen 's Park 3 1 1 1 6 8 4
  FN: Queen 's Park
Sentence: Montrose 3 1 0 2 3 4 3
  FN: Montrose
Sentence: Inverness Thistle 3 1 0 2 3 6 3
  FN: Inverness Thistle
Sentence: East Stirling 3 0 2 1 3 4 2
  FN: East Stirling
Sen

Sentence: SQUASH - HILL BRANDS WORLD CHAMPION JANSHER A CHEAT .
  FN: JANSHER
  FN: HILL
Sentence: Controversial Australian Anthony Hill called Jansher Khan a cheat during his acrimonious defeat by the world number one in the Hong Kong Open semifinals on Saturday .
  FN: Hong Kong Open
  FN: Australian
Sentence: Jansher said that he was disturbed to be called a cheat . "
  FN: Jansher
Sentence: " I think the Professional Squash Association should look into this matter and deal with it properly .
  FP: the Professional Squash Association
  FN: Professional Squash Association
Sentence: Jansher , bidding for an eighth Hong Kong Open title , plays second-seeded Australian Rodney Eyles in the final .
  FP: Hong Kong
  FP: Australian Rodney Eyles
  FN: Jansher
  FN: Hong Kong Open
  FN: Australian
  FN: Rodney Eyles
Sentence: Eyles played his best squash of the tournament to beat fourth-seeded Peter Nicol of Scotland 15-10 8-15 15-10 15-4 .
  FN: Eyles
Sentence: Eyles , who defeated Jansher 

Sentence: SOCCER - RESULT IN SPANISH FIRST DIVISION .
  FP: SPANISH FIRST DIVISION
  FN: SPANISH
Sentence: MADRID 1996-08-31
  FN: MADRID
Sentence: Spanish first division on Saturday :
  FN: division
Sentence: Deportivo Coruna 1 Real Madrid 1
  FN: Real Madrid
  FN: Deportivo Coruna
Sentence: SOCCER - BELGIUM BEAT TURKEY 2-1 IN WORLD CUP QUALIFIER .
  FP: WORLD CUP QUALIFIER
  FN: WORLD CUP
  FN: BELGIUM
  FN: TURKEY
Sentence: Turkey - Sergen Yalcin ( 61st )
  FN: Sergen Yalcin
Sentence: SOCCER - AUSTRIA DRAW 0-0 WITH SCOTLAND IN WORLD CUP QUALIFIER .
  FN: AUSTRIA
  FN: WORLD CUP
  FN: SCOTLAND
Sentence: BOXING - KNOCK-OUT SPECIALIST MILLER DEFENDS TITLE .
  FN: MILLER
Sentence: DUBLIN 1996-08-31
  FN: DUBLIN
Sentence: Heath did score with two brusing lefts to Miller 's head in the third round but failed to put his opponent under any real pressure .
  FN: Heath
Sentence: SOCCER - DUTCH DRAW 2-2 WITH BRAZIL IN FRIENDLY INTERNATIONAL .
  FN: DUTCH
  FN: BRAZIL
Sentence: The Netherlands 

Sentence: A London-based spokesman of the Iraqi National Congress said Iraqi artillery was shelling the city and Iraqi tanks had advanced to within 10 km ( six miles ) of Arbil , the administrative centre of the Kurdish rebel-controlled region of northern Iraq .
  FP: 10 km ( six miles
  FP: the Iraqi National Congress
  FN: London-based
  FN: Iraqi National Congress
Sentence: " At 4.50 a.m. Iraq time ( 0050 GMT ) Iraqi forces began an artillery attack on the outskirts of Arbil , " the spokesman , who asked not to be identified , told Reuters in a telephone call .
  FN: GMT
Sentence: On Thursday , Iraq accused Iran of aggression and said it reserved the right to retaliate for Tehran 's alleged deployment of troops into northern Iraq , where fighting broke out between the two main Iraqi Kurdish rebel groups two weeks ago .
  FP: Iraqi
  FP: Kurdish
  FN: Iraqi Kurdish
Sentence: The report , monitored by the British Broadcasting Corporation ( BBC ) , quoted security services as saying si

Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the *predicted* start and end positions for each token span, rather than the gold positions. As the `label` of each span, you can use the special value `--NME--`.

In [22]:
# TODO: Write code here to store the predicted spans in a new data frame
import numpy as np

def generate_new_df(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    return:
        panda DF with ID, Sentence, token start, token end, and label
    """
    id_list = []
    sentence_list = []
    start_list = []
    end_list = []
    label_list = []
    
    for row in df.itertuples():
        doc = nlp(row[2])
        if row[1] not in id_list:
            for ent in doc.ents:
                if ent.label_ != 'CARDINAL' and ent.label_ != 'DATE' and ent.label_ != 'TIME' and ent.label_ != 'ORDINAL' and ent.label_ != 'PERCENT':
                    id_list.append(row[1])
                    sentence_list.append(row[2])
                    start_list.append(ent.start)
                    end_list.append(ent.end)
                    label_list.append('--NME--')
    return pd.DataFrame(data={'sentence_id': id_list, 
                              'sentence': sentence_list,
                              'beg': start_list,
                              'end': end_list,
                              'label': label_list})


In [23]:
new_df_dev = generate_new_df(df_dev)

In [24]:
new_df_dev.head(20)

sentence_id                                           sentence  beg  end  \
0     0946-001                                  LONDON 1996-08-30    0    1   
1     0946-002  West Indian all-rounder Phil Simmons took four...    0    2   
2     0946-002  West Indian all-rounder Phil Simmons took four...    3    5   
3     0946-002  West Indian all-rounder Phil Simmons took four...   12   13   
4     0946-003  Their stay on top , though , may be short-live...   13   14   
5     0946-003  Their stay on top , though , may be short-live...   15   16   
6     0946-003  Their stay on top , though , may be short-live...   17   18   
7     0946-003  Their stay on top , though , may be short-live...   24   25   
8     0946-003  Their stay on top , though , may be short-live...   35   36   
9     0946-004  After bowling Somerset out for 83 on the openi...    2    3   
10    0946-004  After bowling Somerset out for 83 on the openi...   11   13   
11    0946-004  After bowling Somerset out for 83 on the openi...   14   15   
12    0946-004  After bowling Somerset out for 83 on the openi...   29   30   
13    0946-004  After bowling Somerset out for 83 on the openi...   31   33   
14    0946-005  Trailing by 213 , Somerset got a solid start t...    4    5   
15    0946-005  Trailing by 213 , Somerset got a solid start t...   14   15   
16    0946-006  Essex , however , look certain to regain their...    0    1   
17    0946-006  Essex , however , look certain to regain their...   12   14   
18    0946-006  Essex , however , look certain to regain their...   15   17   
19    0946-006  Essex , however , look certain to regain their...   26   27   

      label  
0   --NME--  
1   --NME--  
2   --NME--  
3   --NME--  
4   --NME--  
5   --NME--  
6   --NME--  
7   --NME--  
8   --NME--  
9   --NME--  
10  --NME--  
11  --NME--  
12  --NME--  
13  --NME--  
14  --NME--  
15  --NME--  
16  --NME--  
17  --NME--  
18  --NME--  
19  --NME--

## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [25]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        yield row[1], row[3], row[4], row[5]  # yield quadruples
    yield None

In [26]:
spans_dev_gold_mentions = set(gold_mentions(df_dev))

In [27]:
spans_dev_gold_mentions

{('1039-013', 0, 2, 'Media_Indonesia'),
 ('0966-164', 19, 21, 'Florian_Schwarthoff'),
 ('1112-002', 13, 15, 'FIFA_World_Cup'),
 ('1077-006', 2, 3, 'Moldova_national_football_team'),
 ('1133-006', 22, 24, 'Amnesty_International'),
 ('1125-008', 3, 6, 'Léon_Kengo_wa_Dondo'),
 ('1010-020', 17, 18, 'United_States'),
 ('0960-014', 12, 13, 'MaliVai_Washington'),
 ('1008-013', 1, 3, 'Dow_Jones_Industrial_Average'),
 ('1033-008', 9, 11, 'Aslan_Maskhadov'),
 ('1055-037', 5, 6, 'Alan_Mullally'),
 ('0960-021', 11, 12, 'Jeff_Tarango'),
 ('1047-003', 2, 4, 'Bradford_Bulls'),
 ('1100-000', 2, 3, '--NME--'),
 ('0961-005', 7, 8, 'Philadelphia_Eagles'),
 ('1030-017', 24, 26, 'Raymond_James_Financial'),
 ('0962-007', 1, 3, 'Bob_Estes'),
 ('1100-002', 1, 3, 'Tom_Johnson_(boxer)'),
 ('1144-002', 0, 3, 'Kurdistan_Democratic_Party'),
 ('0957-026', 0, 2, 'Andriy_Medvedev'),
 ('1016-014', 0, 1, '--NME--'),
 ('1114-002', 10, 11, '--NME--'),
 ('0966-076', 4, 5, 'Kenya'),
 ('0956-009', 0, 1, 'Hanwha_Eagles'),
 (

A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

<div class="alert alert-warning">
    Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.
</div>

In [28]:
# TODO: Write code here to implement the baseline
def naive_pred(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        new_t = row[2]
        new_t = new_t.split(' ')
        yield row[1], row[3], row[4], "_".join(new_t[row[3]:row[4]])  # yield quadruples
    yield None

In [29]:
dev_naive_pred_new = set(naive_pred(new_df_dev))

In [30]:
dev_naive_pred_new

{('1046-001', 0, 1, 'LA'),
 ('0966-164', 19, 21, 'Florian_Schwarthoff'),
 ('1010-018', 0, 1, 'Bernardin'),
 ('1133-006', 22, 24, 'Amnesty_International'),
 ('0967-000', 8, 10, 'CLUB_FINED'),
 ('1059-004', 29, 31, 'Tunbridge_Wells'),
 ('1076-000', 0, 4, 'SOCCER_-_YUGOSLAV_LEAGUE'),
 ('1033-008', 9, 11, 'Aslan_Maskhadov'),
 ('0986-006', 14, 15, 'Houghton'),
 ('1119-007', 7, 8, 'French'),
 ('1155-015', 8, 9, 'German'),
 ('0982-001', 0, 1, 'LONDON'),
 ('1103-015', 74, 79, '18_-_Gilles_De_Bilde'),
 ('0962-007', 1, 3, 'Bob_Estes'),
 ('1058-015', 0, 1, 'Texas'),
 ('1144-002', 0, 3, 'Kurdistan_Democratic_Party'),
 ('1054-017', 4, 5, 'U.S.'),
 ('0994-008', 18, 20, 'Sergei_Posukhov'),
 ('1096-022', 1, 2, 'Braves'),
 ('0966-076', 4, 5, 'Kenya'),
 ('1152-013', 9, 10, 'Iraqi'),
 ('1076-042', 0, 1, 'Zeleznik'),
 ('1005-006', 32, 33, 'Seoul'),
 ('0984-013', 6, 11, 'the_Chicago_Board_of_Trade'),
 ('1007-014', 23, 25, 'John_Geraghty'),
 ('1055-019', 7, 9, 'Mushtaq_Ahmed'),
 ('1118-002', 1, 3, 'Labour_P

In [31]:
evaluation_report(spans_dev_gold_mentions, dev_naive_pred_new)

The precision is: 0.3042697525937749 
 The recall is: 0.25768840824602907 
 The F1-value is: 0.28


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on) as mentions of the entities (pages) that they link to. This allows us to harvest long lists of mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [32]:
with bz2.open('kb.tsv.bz2', 'rt') as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [33]:
df_kb.loc[df_kb.mention == 'Sweden']

mention                                 entity      prob
17436  Sweden                                 Sweden  0.985768
17437  Sweden          Sweden_national_football_team  0.014173
17438  Sweden  Sweden_men's_national_ice_hockey_team  0.000059

As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing come first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [34]:
# TODO: Write code here to implement the "most probable entity" method.
def mark_labels(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    return:
        Return DF with label - Most Probable Entity Link
    """
    id_list = []
    sentence_list = []
    start_list = []
    end_list = []
    label_list = []
    
    for row in df.itertuples():
        sent_text = row[2]
        sent_text = sent_text.split()
        
        id_list.append(row[1])
        sentence_list.append(row[2])
        start_list.append(row[3])
        end_list.append(row[4])
        if len(df_kb.loc[df_kb.mention == " ".join(sent_text[row[3]:row[4]])]) != 0:
            entity_data = df_kb.loc[df_kb.mention == " ".join(sent_text[row[3]:row[4]])]
            label_list.append(entity_data['entity'].tolist()[0])
        else:
            label_list.append('--NME--')
    return pd.DataFrame(data={'sentence_id': id_list, 
                              'sentence': sentence_list,
                              'beg': start_list,
                              'end': end_list,
                              'label': label_list})

In [35]:
new_labeled_df_dev = mark_labels(new_df_dev)

In [36]:
new_labeled_df_dev.head(20)

sentence_id                                           sentence  beg  end  \
0     0946-001                                  LONDON 1996-08-30    0    1   
1     0946-002  West Indian all-rounder Phil Simmons took four...    0    2   
2     0946-002  West Indian all-rounder Phil Simmons took four...    3    5   
3     0946-002  West Indian all-rounder Phil Simmons took four...   12   13   
4     0946-003  Their stay on top , though , may be short-live...   13   14   
5     0946-003  Their stay on top , though , may be short-live...   15   16   
6     0946-003  Their stay on top , though , may be short-live...   17   18   
7     0946-003  Their stay on top , though , may be short-live...   24   25   
8     0946-003  Their stay on top , though , may be short-live...   35   36   
9     0946-004  After bowling Somerset out for 83 on the openi...    2    3   
10    0946-004  After bowling Somerset out for 83 on the openi...   11   13   
11    0946-004  After bowling Somerset out for 83 on the openi...   14   15   
12    0946-004  After bowling Somerset out for 83 on the openi...   29   30   
13    0946-004  After bowling Somerset out for 83 on the openi...   31   33   
14    0946-005  Trailing by 213 , Somerset got a solid start t...    4    5   
15    0946-005  Trailing by 213 , Somerset got a solid start t...   14   15   
16    0946-006  Essex , however , look certain to regain their...    0    1   
17    0946-006  Essex , however , look certain to regain their...   12   14   
18    0946-006  Essex , however , look certain to regain their...   15   17   
19    0946-006  Essex , however , look certain to regain their...   26   27   

                                  label  
0                                London  
1              West_Indies_cricket_team  
2                          Phil_Simmons  
3    Leicestershire_County_Cricket_Club  
4                                 Essex  
5        Derbyshire_County_Cricket_Club  
6                                Surrey  
7              Kent_County_Cricket_Club  
8   Nottinghamshire_County_Cricket_Club  
9          Somerset_County_Cricket_Club  
10                           Grace_Road  
11   Leicestershire_County_Cricket_Club  
12                              England  
13                       Andrew_Caddick  
14         Somerset_County_Cricket_Club  
15                              --NME--  
16                                Essex  
17                       Nasser_Hussain  
18                           Peter_Such  
19                            Yorkshire

In [37]:
df_dev_pred_mentions = set(gold_mentions(new_labeled_df_dev))

In [38]:
df_dev_pred_mentions

{('0966-164', 19, 21, 'Florian_Schwarthoff'),
 ('1112-002', 13, 15, 'FIFA_World_Cup'),
 ('1133-006', 22, 24, 'Amnesty_International'),
 ('1125-008', 3, 6, 'Léon_Kengo_wa_Dondo'),
 ('1033-008', 9, 11, 'Aslan_Maskhadov'),
 ('0960-021', 11, 12, 'Jeff_Tarango'),
 ('0961-005', 7, 8, 'Philadelphia_Eagles'),
 ('1135-025', 31, 32, 'Insein_Prison'),
 ('0962-007', 1, 3, 'Bob_Estes'),
 ('1069-013', 25, 28, '--NME--'),
 ('1058-015', 0, 1, 'Texas'),
 ('0957-026', 0, 2, 'Andriy_Medvedev'),
 ('1144-002', 0, 3, 'Kurdistan_Democratic_Party'),
 ('1113-005', 0, 5, '--NME--'),
 ('1016-014', 0, 1, '--NME--'),
 ('0966-076', 4, 5, 'Kenya'),
 ('0958-017', 0, 1, '--NME--'),
 ('1005-006', 32, 33, 'Seoul'),
 ('1121-003', 22, 27, '--NME--'),
 ('1126-001', 2, 4, '--NME--'),
 ('1061-017', 0, 1, 'Scotland'),
 ('1056-037', 0, 2, 'Florian_Rousseau'),
 ('1007-025', 18, 19, 'Whitewater_controversy'),
 ('1051-005', 5, 6, 'Spain'),
 ('1060-020', 1, 3, '--NME--'),
 ('1064-007', 20, 24, '--NME--'),
 ('1104-002', 6, 7, 'Span

In [39]:
evaluation_report(spans_dev_gold_mentions, df_dev_pred_mentions)

The precision is: 0.6183160415003991 
 The recall is: 0.5236566407570125 
 The F1-value is: 0.57


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier that is trained to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [40]:
with bz2.open('contexts.tsv.bz2') as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [41]:
df_contexts.head()

mention                 entity  \
0            1970  UEFA_Champions_League   
1            1970         FIFA_World_Cup   
2  1990 World Cup    1990_FIFA_World_Cup   
3  1990 World Cup    1990_FIFA_World_Cup   
4  1990 World Cup    1990_FIFA_World_Cup   

                                             context  
0    Cup twice the first in @ and the second in 1983  
1  America 1975 and during the @ and 1978 World C...  
2                 Manolo represented Spain at the @   
3  Hašek represented Czechoslovakia at the @ and ...  
4  renovations in 1989 for the @ The present capa...

Note that, in each context, the position of the mention is indicated by the `@` symbol.

From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [42]:
df_contexts.context[df_contexts.mention == 'Lincoln']

41465    Nebraska Concealed Handgun Permit In @ municip...
41466    Lazlo restaurants are located in @ and Omaha C...
41467    California Washington Overland Park Kansas @ N...
41468    City Missouri Omaha Nebraska and @ Nebraska It...
41469    by Sandhills Publishing Company in @ Nebraska USA
                               ...                        
41609                                      @ Leyton Orient
41610                    English division three Swansea @ 
41611    league membership narrowly edging out @ on goa...
41612                                          @ Cambridge
41613                                                   @ 
Name: context, Length: 149, dtype: object

In [43]:
df_contexts[df_contexts.mention == 'Lincoln']

mention             entity  \
41465  Lincoln  Lincoln,_Nebraska   
41466  Lincoln  Lincoln,_Nebraska   
41467  Lincoln  Lincoln,_Nebraska   
41468  Lincoln  Lincoln,_Nebraska   
41469  Lincoln  Lincoln,_Nebraska   
...        ...                ...   
41609  Lincoln  Lincoln_City_F.C.   
41610  Lincoln  Lincoln_City_F.C.   
41611  Lincoln  Lincoln_City_F.C.   
41612  Lincoln  Lincoln_City_F.C.   
41613  Lincoln  Lincoln_City_F.C.   

                                                 context  
41465  Nebraska Concealed Handgun Permit In @ municip...  
41466  Lazlo restaurants are located in @ and Omaha C...  
41467  California Washington Overland Park Kansas @ N...  
41468  City Missouri Omaha Nebraska and @ Nebraska It...  
41469  by Sandhills Publishing Company in @ Nebraska USA  
...                                                  ...  
41609                                    @ Leyton Orient  
41610                  English division three Swansea @   
41611  league membership narrowly edging out @ on goa...  
41612                                        @ Cambridge  
41613                                                 @   

[149 rows x 3 columns]

Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [44]:
# # TODO: Write code here to implement the context-sensitive disambiguation method
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import CountVectorizer

# def mark_labels(df):
#     """Yield the gold-standard mentions in a data frame.

#     Args:
#         df: A data frame.

#     return:
#         Return DF with label - Most Probable Entity Link
#     """
#     id_list = []
#     sentence_list = []
#     start_list = []
#     end_list = []
#     label_list = []
    
#     for row in df.itertuples():
#         men_text = row[5]
#         men_text = men_text.split()
#         men_text = " ".join(men_text)
        
#         id_list.append(row[1])
#         sentence_list.append(row[2])
#         start_list.append(row[3])
#         end_list.append(row[4])
        
#         mentioned_data = df_contexts[df_contexts.mention == men_text]
        
#         if len(mentioned_data) != 0:
#             if len(mentioned_data['entity'].tolist()) != 1:
# #                 df_kb.loc[df_kb.mention == 'Sweden']
#                 entity_tags = df_kb.loc[df_kb.mention == row[5]]
#                 prior_prob = entity_tags.sort_values('entity', ascending=True)['prob'].tolist()
                
#                 pipe = Pipeline([('count_vec',CountVectorizer()),('MNB', MultinomialNB(class_prior=prior_prob))])
#                 pipe.fit(df_contexts[df_contexts.mention == men_text]['context'], 
#                          df_contexts[df_contexts.mention == men_text]['entity'].tolist())
#                 y_pred = pipe.predict([row[2]])
#                 label_list.append(y_pred[0])
            
#             else:
#                 label_list.append(mentioned_data['entity'].tolist()[0])
#         else:
#             label_list.append(row[5])
#     return pd.DataFrame(data={'sentence_id': id_list, 
#                               'sentence': sentence_list,
#                               'beg': start_list,
#                               'end': end_list,
#                               'label': label_list})

In [46]:
# TODO: Write code here to implement the context-sensitive disambiguation method
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

def mark_labels(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    return:
        Return DF with label - Most Probable Entity Link
    """
    id_list = []
    sentence_list = []
    start_list = []
    end_list = []
    label_list = []
    
    # Save pipline for new tags, and reuse when needed
    pip_dic = {}
    
    for row in df.itertuples():
        men_text = row[5]
        men_text = men_text.split()
        men_text = " ".join(men_text)
        
        id_list.append(row[1])
        sentence_list.append(row[2])
        start_list.append(row[3])
        end_list.append(row[4])
        
        mentioned_data = df_contexts[df_contexts.mention == men_text]
        
        if len(mentioned_data) != 0:
            if len(mentioned_data['entity'].tolist()) != 1:
#                 df_kb.loc[df_kb.mention == 'Sweden']
                pred_pipe = None
                if row[5] in pip_dic:
                    # NB already initialised and available on pip_dic
                    pass
                else:
                    # NB not initialised and save on pip_dic
                    entity_tags = df_kb.loc[df_kb.mention == row[5]]
                    prior_prob = entity_tags.sort_values('entity', ascending=True)['prob'].tolist()
                
                    pipe = Pipeline([('count_vec',CountVectorizer()),('MNB', MultinomialNB(class_prior=prior_prob))])
                    pipe.fit(df_contexts[df_contexts.mention == men_text]['context'], 
                             df_contexts[df_contexts.mention == men_text]['entity'].tolist())
                    # Save pipe for later use
                    pip_dic[row[5]] = pipe
                
                # Load Pipe from Dictionary
                pred_pipe = pip_dic[row[5]]
            
                y_pred = pred_pipe.predict([row[2]])
                label_list.append(y_pred[0])
            
            else:
                label_list.append(mentioned_data['entity'].tolist()[0])
        else:
            label_list.append(row[5])
    return pd.DataFrame(data={'sentence_id': id_list, 
                              'sentence': sentence_list,
                              'beg': start_list,
                              'end': end_list,
                              'label': label_list})

In [47]:
df_dev_new_labeled = mark_labels(new_labeled_df_dev)

In [48]:
df_dev_new_labeled.head()

sentence_id                                           sentence  beg  end  \
0    0946-001                                  LONDON 1996-08-30    0    1   
1    0946-002  West Indian all-rounder Phil Simmons took four...    0    2   
2    0946-002  West Indian all-rounder Phil Simmons took four...    3    5   
3    0946-002  West Indian all-rounder Phil Simmons took four...   12   13   
4    0946-003  Their stay on top , though , may be short-live...   13   14   

                                label  
0                              London  
1            West_Indies_cricket_team  
2                        Phil_Simmons  
3  Leicestershire_County_Cricket_Club  
4           Essex_County_Cricket_Club

In [49]:
dev_pred_mentions = set(gold_mentions(df_dev_new_labeled))

In [50]:
dev_pred_mentions

{('0966-164', 19, 21, 'Florian_Schwarthoff'),
 ('1112-002', 13, 15, 'FIFA_World_Cup'),
 ('1077-006', 2, 3, 'Moldova_national_football_team'),
 ('1133-006', 22, 24, 'Amnesty_International'),
 ('1125-008', 3, 6, 'Léon_Kengo_wa_Dondo'),
 ('1033-008', 9, 11, 'Aslan_Maskhadov'),
 ('0960-021', 11, 12, 'Jeff_Tarango'),
 ('0961-005', 7, 8, 'Philadelphia_Eagles'),
 ('1135-025', 31, 32, 'Insein_Prison'),
 ('0962-007', 1, 3, 'Bob_Estes'),
 ('1069-013', 25, 28, '--NME--'),
 ('1058-015', 0, 1, 'Texas'),
 ('0957-026', 0, 2, 'Andriy_Medvedev'),
 ('1144-002', 0, 3, 'Kurdistan_Democratic_Party'),
 ('1113-005', 0, 5, '--NME--'),
 ('1016-014', 0, 1, '--NME--'),
 ('0966-076', 4, 5, 'Kenya'),
 ('0958-017', 0, 1, '--NME--'),
 ('1005-006', 32, 33, 'Seoul'),
 ('1121-003', 22, 27, '--NME--'),
 ('1126-001', 2, 4, '--NME--'),
 ('1061-017', 0, 1, 'Scotland'),
 ('1056-037', 0, 2, 'Florian_Rousseau'),
 ('1007-025', 18, 19, 'Whitewater_controversy'),
 ('1051-005', 5, 6, 'Spain'),
 ('1060-020', 1, 3, '--NME--'),
 ('1

In [51]:
evaluation_report(spans_dev_gold_mentions, dev_pred_mentions)

The precision is: 0.6233040702314445 
 The recall is: 0.5278810408921933 
 The F1-value is: 0.57


You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1.

**This was the last lab in the Text Mining course. Congratulations!**

<div class="alert alert-info">
    Please read the section ‘General information’ on the ‘Labs’ page of the course website before submitting this notebook!
</div>